In [ ]:
'''
ZCZC
Kawasaki Fast-MeteoERA Toolkits (CMA-BST Edition Version0.5Beta)

使用前建议新建虚拟环境，安装所需依赖包
建议在带梯子的环境下运行此脚本，以确保能够顺利下载 ERA-5数据 或者 地理信息 数据
github.com/KawasakiKusako


@Kawasaki Kusako
2025-11-26

BABJ-CDUT
NNNN

'''

from function.dataRead import read_typhoon_data, get_typhoon_list
from function.getERA import choose_typhoon, get_ERA_data, select_ERA_data

import pandas as pd

#设置参数

#请确认是否存在该年份台风数据在 dataSet 文件夹内

#输入你想要计算的年份，原始数据保存至dataSet/CH{inputYear}BST.txt
inputYear = int("1994")



In [ ]:
# 选择台风
df = read_typhoon_data(inputYear)
get_typhoon_list(inputYear, df)


typhoon_list = pd.read_csv(f"cache/typhoon_list_{inputYear}.csv")

start_time, end_time, typhoon_name, typhoon_id, output_path = choose_typhoon(typhoon_list, inputYear)
print(f"你选择的是：{start_time} 至 {end_time} 的 {typhoon_name}（{typhoon_id}）")
print(output_path)

In [ ]:
#处理单个台风
from yearlyAnaly.preAnaly import generate_typhoon_csv_by_year
generate_typhoon_csv_by_year(str(inputYear))

In [ ]:
# 可视化分析路径
from yearlyAnaly.trackViz import load_typhoon_data, plot_typhoon_track, plot_all_typhoons_track, load_all_typhoon_data_for_year

#单一台风可视化（自上面的选择）
plot_typhoon_track(typhoon_dict=load_typhoon_data(inputYear, typhoon_id, typhoon_name), dpi=300)
#年度可视化（自上面的选择）
plot_all_typhoons_track(all_typhoon_data=load_all_typhoon_data_for_year(inputYear), inputYear=inputYear, dpi=300)


In [ ]:
# 时空演变分析
from yearlyAnaly.sptiAnaly import analyze_co_evolution
from yearlyAnaly.sptiAnaly import plot_cluster_distribution_analysis

# 预设聚类数为？类（根据之前分析，3~4类聚类效果较好）
K_CLUSTERS = 3

# 1. 加载全年所有台风数据

all_typhoon_data = load_all_typhoon_data_for_year(inputYear)

features_df_with_clusters, cluster_summary_df = analyze_co_evolution(
        all_typhoon_data=all_typhoon_data, 
        inputYear=inputYear, 
        n_clusters=K_CLUSTERS,
        dpi=300
    )

plot_cluster_distribution_analysis(
        features_df=features_df_with_clusters, # 传入包含 'Cluster' 列的 DataFrame
        inputYear=inputYear,
        n_clusters=K_CLUSTERS,
        dpi=300
    )
display(cluster_summary_df) # 使用 display() 以美观地显示 DataFrame

In [ ]:
#台风时间分析
from yearlyAnaly.temporalAnaly import (
    plot_monthly_formation_distribution,
    plot_lifetime_distribution,
    plot_temporal_coordination_heatmap
)
all_typhoon_data = load_all_typhoon_data_for_year(inputYear)
    
#月度生成频率分析
plot_monthly_formation_distribution(
        all_typhoon_data=all_typhoon_data,
        inputYear=inputYear,
        dpi=300
    )
    
#生命期分布分析
plot_lifetime_distribution(
        all_typhoon_data=all_typhoon_data,
        inputYear=inputYear,
        dpi=300
    )
#时间协同热力图分析
plot_temporal_coordination_heatmap(
        all_typhoon_data=all_typhoon_data,
        inputYear=inputYear,
        dpi=300
    )
    

In [ ]:
# 下载单一台风对应时间点的 ERA5 数据
# 下载前请确保已经登录 CDS 网站 并 配置好 cdsapi 模块（https://cds.climate.copernicus.eu/how-to-api）
# Windows 请 查找 C:\Users\你的用户名\.cdsapirc 文件 进行配置

# 时间需求较长，需要耐心等待
get_ERA_data(start_time, end_time, inputYear, typhoon_name)

# 若下载失败 则 下载 当年所有数据(不要使用此功能，除非必要！！！！！！！！！！！！！！！)
#from function.wholeERA import get_ERA_yearly_data
#get_ERA_yearly_data(inputYear)

#你可以去https://cds.climate.copernicus.eu/requests?tab=all
#检查request是否成功


In [ ]:
# ===================   单一台风详细分析  =======================

# 初始场 数据 可视化
start_time_str = str(start_time)[:8] + '00'
end_time_str = str(end_time)[:8] + '00'

print(start_time_str)
print(end_time_str)


from pyViz.advanceVizTypeA import plot_typeA
from pyViz.advanceVizTypeA100 import plot_typeA100
from pyViz.advanceVizTypeB import plot_typeB
from pyViz.advanceVizTypeC import plot_typeC
from pyViz.advanceMeteoVizSWEAT import plot_typeS

# 以下是几种常用的台风分析图（用于气象分析）
# 850hPa温度+风矢量  与  500hPa高度场 or 100hPa
plot_typeA(start_time_str, end_time_str, inputYear, typhoon_name)
plot_typeA100(start_time_str, end_time_str, inputYear, typhoon_name)
# MSLP 与 850hPa 风矢量 涡度
plot_typeB(start_time_str, end_time_str, inputYear, typhoon_name)
# 500hPa 相对湿度 与 200hPa 风矢量
plot_typeC(start_time_str, end_time_str, inputYear, typhoon_name)
# SWEAT 指数
plot_typeS(start_time_str, end_time_str, inputYear, typhoon_name)



In [ ]:
# 台风路径预测（Pangu-Weather-ReadyToGo）
# 需要自行安装 Pangu-Weather-ReadyToGo 环境
# 在此不做演示，仅作提示。